In [24]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2

# Loading dataset

In [22]:
def get_data():
    URL = 'data/'

    imgs = os.listdir(URL)

    return [cv2.imread(f"{URL}{img}") for img in imgs]



In [23]:
test = get_data()